In [1]:
%load_ext autoreload
%autoreload 2

In [74]:
import os
from collections import namedtuple

import matplotlib.pyplot as plt 
import cv2 
import json
import numpy as np

import ritnet
import nia22
from glob import glob

from nia22 import scenario
from nia22.mask_utils import gen_mask
from nia22.eyes import Eye, device_dict, draw_two_eyes

In [3]:
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [4]:
ddir = '/home/hoseung/Work/NIA/data/jsons/'
vid_dir_base = '/home/hoseung/Work/NIA/data/run/by_id/'
#ddir = "/home/di/Work/data1/NIA2022/ORG/"
#vid_dir_base = "/mnt/syn2422/raw/raw/by_id/"

In [73]:
def get_all_scenario(uid, devices = ["V", "M"]):
    if not isinstance(uid, str): uid = f"{uid:03d}"
    pattern = ddir + f"{uid}/*.json"
    json_list = glob(pattern)
    json_list.sort()
    print(f"{len(json_list)} JSON files are found")
    
    ####
    all_scenarios = []
    for jl in json_list:
        jj = jl.split("/")[-1]
        device = jj.split("_")[5]
        code = jj[-14:-9]
        if device in devices:
        #if True:
            all_scenarios.append((device, code))

    scenario_set = set(all_scenarios)
    return scenario_set



RGB -> IR

기기에 따라 affine transformation 찾아내기.
affine transformation은 뭘로 찾을까?
ORB나 SWIFT feature 대신 mediapipe mesh끼리 비교. 


끝. 



In [11]:
# 1. json 중 RGB와 IR 영상이 모두 있는 리스트

# 2. RGB의 json과 face mesh 비교 




In [6]:
uid = 11
scenario_set = get_all_scenario(uid)

for device, ss in scenario_set:
    print("Device", device_dict[device], "Scenario", ss)
    file = scenario.Info(device, base_dir = ddir + f"{uid:03d}/",
                         scen = ss)
    file.vid_dir = vid_dir_base + f"{file.id}/T1/{file.device_d}/RGB/"
    fn_vid = file.vid_dir + file.fn_vid
    
    break

10070 JSON files are found
Device Monitor Scenario F_H_T


In [13]:
if (cap_r := file.load_vid('rgb')) and (cap_i := file.load_vid('ir')):
    for ff, json in file:
        # read specific frame 
        cap_r.set(cv2.CAP_PROP_POS_FRAMES, ff)
        ok_rgb, rgb = cap_r.read()
        #rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        #rgb = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
        cap_i.set(cv2.CAP_PROP_POS_FRAMES, ff)
        ok_ir, ir = cap_i.read()
        
        # IR to greyscale 혹은 R 밴드만? 
        ir = cv2.cvtColor(ir, cv2.COLOR_BGR2GRAY)

        eye = file.get_eye(json)
        # then do something1111
        plot_eyes(rgb, eye, f"{file.id}_{ff:03d}_rgb.png")
        plot_eyes(ir, eye, f"{file.id}_{ff:03d}_ir.png")
        
        break
else:
    print("Failed to open both videos")
    
    

In [75]:
ok_rgb, rgb = cap_r.read()

In [77]:
image = draw_two_eyes(rgb, eye)
# Displaying the image
while(1):
    cv2.imshow('image', image)
    if cv2.waitKey(20) & 0xFF == 27:
        break
         
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x55fa78cee470) is not the object's thread (0x55fa76057a90).
Cannot move to target thread (0x55fa78cee470)

QObject::moveToThread: Current thread (0x55fa78cee470) is not the object's thread (0x55fa76057a90).
Cannot move to target thread (0x55fa78cee470)

QObject::moveToThread: Current thread (0x55fa78cee470) is not the object's thread (0x55fa76057a90).
Cannot move to target thread (0x55fa78cee470)

QObject::moveToThread: Current thread (0x55fa78cee470) is not the object's thread (0x55fa76057a90).
Cannot move to target thread (0x55fa78cee470)

QObject::moveToThread: Current thread (0x55fa78cee470) is not the object's thread (0x55fa76057a90).
Cannot move to target thread (0x55fa78cee470)

QObject::moveToThread: Current thread (0x55fa78cee470) is not the object's thread (0x55fa76057a90).
Cannot move to target thread (0x55fa78cee470)

QObject::moveToThread: Current thread (0x55fa78cee470) is not the object's thread (0x55fa76057a90).
Cannot move to tar

## 2. JSON 로딩 

In [ ]:
anno = json.load(open(file._dir + fn_json,"r"))

## 3. RGB / IR 비교

In [25]:
eye = Eye(anno["Annotations"]["annotations"])

## 4. RGB -> IR 변환